In [1]:
import numpy as np

from sudoku_stuff import *

# Resolviendo Sudokus con gradiente descendiente
Inteligencia Artificial - Facundo A. Lucianna - CEIA - FIUBA

Dado que ya tenemos implementado los elementos para resolver Sudokus, resolvamos primero con gradiente descendiente discreto.

Resolvamos este problema:

<div>
<img src="./sudoku_7.png" width="300"/>
</div>

El diccionario de **celdas fijas** quedaría de la siguiente forma:

In [2]:
fixed_squares = {
    'A1': 3, 'A3': 4, 'A4': 5, 'A5': 6, 'A7': 9,
    'B1': 1, 'B2': 8, 'B3': 5, 'B6': 9, 'B7': 7,
    'C5': 7, 'C6': 8, 'C7': 4, 'C8': 1, 'C9': 5,
    'D2': 2, 'D5': 1, 'D8': 4, 'D9': 9,
    'E2': 4, 'E3': 9, 'E5': 5, 
    'F3': 1, "F4": 9, "F5": 8, "F7": 6, "F8": 7,
    'G1': 4, 'G2': 9, 'G5': 3, 'G9': 7, 
    'H2': 1, 'H3': 8, 'H4': 7, 'H5': 4, 'H6': 5, 'H9': 6,
    'I8': 8,
}

Y la solución que tenemos, que vamos a usar para verificar al final de todo es:

In [3]:
solution = {
    'A1': 3, 'A2': 7, 'A3': 4, 'A4': 5, 'A5': 6, 'A6': 1, 'A7': 9, 'A8': 2, 'A9': 8,
    'B1': 1, 'B2': 8, 'B3': 5, 'B4': 4, 'B5': 2, 'B6': 9, 'B7': 7, 'B8': 6, 'B9': 3,
    'C1': 9, 'C2': 6, 'C3': 2, 'C4': 3, 'C5': 7, 'C6': 8, 'C7': 4, 'C8': 1, 'C9': 5,
    'D1': 8, 'D2': 2, 'D3': 7, 'D4': 6, 'D5': 1, 'D6': 3, 'D7': 5, 'D8': 4, 'D9': 9,
    'E1': 6, 'E2': 4, 'E3': 9, 'E4': 2, 'E5': 5, 'E6': 7, 'E7': 8, 'E8': 3, 'E9': 1,
    'F1': 5, 'F2': 3, 'F3': 1, 'F4': 9, 'F5': 8, 'F6': 4, 'F7': 6, 'F8': 7, 'F9': 2,
    'G1': 4, 'G2': 9, 'G3': 6, 'G4': 8, 'G5': 3, 'G6': 2, 'G7': 1, 'G8': 5, 'G9': 7,
    'H1': 2, 'H2': 1, 'H3': 8, 'H4': 7, 'H5': 4, 'H6': 5, 'H7': 3, 'H8': 9, 'H9': 6,
    'I1': 7, 'I2': 5, 'I3': 3, 'I4': 1, 'I5': 9, 'I6': 6, 'I7': 2, 'I8': 8, 'I9': 4,
}

In [4]:
print_state(solution)

*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*


## Implementado gradiente descendente

Este algoritmo se mueve continuamente en la dirección de mayor decrecimiento del valor de la función de costo. La búsqueda termina cuando ningún vecino tiene un valor menor. El algoritmo no examina más allá de los vecinos inmediatos.

Veamos la implementación, leyendo cada comentario con atención:

In [5]:
def gradient_descent_sudoku(initial_state: dict, fixed_squares: dict, max_iterations: int = 1000) -> tuple:
    """
    Realiza la optimización del Sudoku utilizando el método de descenso de gradiente.

    Args:
        initial_state (dict): El estado inicial del Sudoku.
        fixed_squares (dict): Diccionario que contiene las casillas fijas del Sudoku,
        max_iterations (int, opcional): El número máximo de iteraciones permitidas.
                                        Por defecto es 100.
    Returns:
        dict: El mejor estado encontrado después de la optimización.
        float: El costo del mejor estado encontrado
    """
    best_state = initial_state
    cost_state = cost_function(best_state)

    # Iteramos hasta max_iterations
    for iteration in range(max_iterations):

        # Calculamos la función de costo para el estado actual
        cost_state = cost_function(best_state)

        # Si el costo es cero, significa que estamos en el minimo. Esto tiene sentido para el caso de Sudoku y la
        # función de costo que implementamos.
        if cost_state == 0:
            break

        # Obtenemos a los vecinos más cercanos
        neib_states = return_neib_states(best_state, fixed_squares)

        # Calculamos el delta del costo entre el estado actual y sus vecinos
        neib_energy_list = [cost_function(neib_state) - cost_state for neib_state in neib_states]

        # Obtenemos el índice de la lista de estados vecinos que tenga el mínimo valor
        index_min_energy = np.argmin(neib_energy_list)

        # Si el delta del costo es positivo o cero, es que no hay vecino que reduzca el gradiente, llegamos a un minimo
        # local.
        if neib_energy_list[index_min_energy] >= 0:
            return best_state, cost_state

        # Si no, seguimos avanzando
        best_state = neib_states[index_min_energy]

    # Si terminamos las iteraciones, retornamos nuestro mejor resultado
    return best_state, cost_state

Ahora, veamos si podemos encontrar la solución en una única ejecución. Para ello, vamos a llamar a una función llamada `execute_search()`, a la cual le pasamos la implementación del algoritmo. Esta función inicializa el sudoku en un estado al azar, aplica la búsqueda y verifica si se ha alcanzado una solución o no.

In [6]:
from processing import execute_search

In [7]:
# El primer número es un identificador que por ahora no nos importa que valor asume.
solution_bool, last_state, initial_state, _ = execute_search(0, gradient_descent_sudoku, fixed_squares)

In [8]:
print(f"Valor de costo inicial {cost_function(initial_state)} y final {cost_function(last_state)}")
print("Primer estado:")
print_state(initial_state)

print("Último estado encontrado:")
print_state(last_state)

Valor de costo inicial 7.1000000000000005 y final 0.7999999999999999
Primer estado:
*---------+---------+---------*
| 3  1  4 | 5  6  5 | 9  9  5 |
| 1  8  5 | 5  7  9 | 7  7  3 |
| 7  2  9 | 8  7  8 | 4  1  5 |
*---------+---------+---------*
| 6  2  9 | 5  1  6 | 5  4  9 |
| 3  4  9 | 2  5  7 | 5  2  7 |
| 5  3  1 | 9  8  5 | 6  7  8 |
*---------+---------+---------*
| 4  9  2 | 7  3  8 | 7  5  7 |
| 5  1  8 | 7  4  5 | 2  2  6 |
| 3  4  8 | 5  9  4 | 6  8  6 |
*---------+---------+---------*
Último estado encontrado:
*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  2  6 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 5  2  7 | 3  1  6 | 8  4  9 |
| 8  4  9 | 2  5  7 | 5  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  2 | 6  3  8 | 1  5  7 |
| 7  1  8 | 7  4  5 | 2  9  6 |
| 6  5  3 | 1  9  2 | 3  8  4 |
*---------+---------+---------*


In [9]:
print("El estado encontrado es solución?")
if solution_bool:
    print("El estado que encontramos verifica que realmente es la solución")
else:
    print("El estado que encontramos no es solución")

El estado encontrado es solución?
El estado que encontramos no es solución


Vemos que el algoritmo no está encontrando la **solución**. Ahora, podemos probar repetir 500 ejecuciones de la busqueda, comenzando desde diferentes puntos iniciales, para ver si eventualmente llegamos a la solución. Para acelerar el proceso, aprovecharemos que tenemos **CPUs multinúcleo**.

Para ello, vamos a llamar a la función `parallel_sudoku_search()`, a la cual le pasaremos la función de gradiente descendente y el número de iteraciones que queremos realizar.

Nota: Deben instalar `tqdm` para poder visualizar el progreso de las ejecuciones. Si están usando conda, pueden instalarlo ejecutando el siguiente comando:

```
conda install -y tqdm
```

In [10]:
from processing import parallel_sudoku_search

# Debemos llamar a la función de busqueda desde un archivo .py sino los threads no pueden recibir a la función desde la notebook directamente.
from search_methods import gradient_descent_sudoku

In [11]:
results = parallel_sudoku_search(gradient_descent_sudoku, fixed_squares, max_iterations=500)

  0%|          | 0/500 [00:00<?, ?it/s]

Veamos si algún proceso encontró la solución:

In [12]:
show_solution = True
for res in results:
    # Acá nos devuelve el booleano de si encontró la solución o no
    is_solution = res[0]
    # Este es el últimos estado encontrado en esta iteración
    last_state = res[1]
    # Este es el estado desde donde partio
    initial_state = res[2]
    # Este es el identificador de cual iteración se obtuvo la solución
    process_id = res[-1]

    if is_solution:
        if show_solution:
            print_state(last_state)
            show_solution = False
        print(f"En la iteración {process_id} se encontró la solución.")

*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*
En la iteración 485 se encontró la solución.


---
## Gradiente descendiente estocastico

Vemos que con el gradiente descendente está siendo difícil encontrar la solución. Una variante de este algoritmo es el gradiente descendente estocástico, en el cual, en lugar de movernos siempre hacia la dirección de la máxima pendiente (o derivada), nos movemos en una dirección aleatoria que aún mantenga el descenso en el valor de la función de costo.

Veamos la implementación, prestando atención a los comentarios en detalle:

In [13]:
def gradient_descent_random_sudoku(initial_state: dict, fixed_squares: dict, max_iterations: int = 1000) -> tuple:
    """
    Realiza la optimización del Sudoku utilizando el método de descenso de gradiente estocástico.

    Args:
        initial_state (dict): El estado inicial del Sudoku.
        fixed_squares (dict): Diccionario que contiene las casillas fijas del Sudoku,
        max_iterations (int, opcional): El número máximo de iteraciones permitidas.
                                        Por defecto es 100.
    Returns:
        dict: El mejor estado encontrado después de la optimización.
        float: El costo del mejor estado encontrado
    """
    best_state = initial_state
    cost_state = cost_function(best_state)

    # Iteramos hasta max_iterations
    for iteration in range(max_iterations):

        # Calculamos la función de costo para el estado actual
        cost_state = cost_function(best_state)

        # Si el costo es cero, significa que estamos en el minimo. Esto tiene sentido para el caso de Sudoku y la
        # función de costo que implementamos.
        if cost_state == 0:
            break

        # Obtenemos a los vecinos más cercanos
        neib_states = return_neib_states(best_state, fixed_squares)

        # Calculamos el delta del costo entre el estado actual y sus vecinos
        neib_energy_list = [cost_function(neib_state) - cost_state for neib_state in neib_states]

        # Obtenemos el índice de la lista de estados vecinos que tengan el mínimo valor y que además sea negativo
        # significando que estamos descendiendo en la función en la dirección de máximo descenso.
        index_min_energy = [i for i, x in enumerate(neib_energy_list) if x < 0]

        # Si no tenemos ningún índice, significa que ya no hay más
        # descenso, retornamos lo mejor que llegó el método
        if not index_min_energy:
            return best_state, cost_state

        # Si no, elegimos una dirección al azar de los máximos cambios
        index_sel = random.choice(index_min_energy)
        best_state = neib_states[index_sel]

    # Si terminamos las iteraciones, retornamos nuestro mejor resultado
    return best_state, cost_state

In [14]:
# El primer número es un identificador que por ahora no nos importa que valor asume.
solution_bool, last_state, initial_state, _ = execute_search(0, gradient_descent_random_sudoku, fixed_squares)

In [15]:
print(f"Valor de costo inicial {cost_function(initial_state)} y final {cost_function(last_state)}")
print("Primer estado:")
print_state(initial_state)

print("Último estado encontrado:")
print_state(last_state)

Valor de costo inicial 7.0 y final 1.0999999999999999
Primer estado:
*---------+---------+---------*
| 3  1  4 | 5  6  9 | 9  8  5 |
| 1  8  5 | 1  5  9 | 7  5  3 |
| 9  8  4 | 2  7  8 | 4  1  5 |
*---------+---------+---------*
| 2  2  7 | 9  1  2 | 8  4  9 |
| 8  4  9 | 2  5  3 | 9  9  2 |
| 8  1  1 | 9  8  7 | 6  7  5 |
*---------+---------+---------*
| 4  9  5 | 3  3  5 | 8  1  7 |
| 1  1  8 | 7  4  5 | 4  2  6 |
| 7  9  1 | 2  6  6 | 9  8  7 |
*---------+---------+---------*
Último estado encontrado:
*---------+---------+---------*
| 3  2  4 | 5  6  7 | 9  6  8 |
| 1  8  5 | 1  4  9 | 7  2  3 |
| 9  7  6 | 2  7  8 | 4  1  5 |
*---------+---------+---------*
| 5  2  7 | 6  1  3 | 8  4  9 |
| 6  4  9 | 7  5  2 | 5  3  1 |
| 8  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  1 | 2  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 9  2  6 | 1  8  4 |
*---------+---------+---------*


In [16]:
print("El estado encontrado es solución?")
if solution_bool:
    print("El estado que encontramos verifica que realmente es la solución")
else:
    print("El estado que encontramos no es solución")

El estado encontrado es solución?
El estado que encontramos no es solución


Veamos ahora ejecutando varias veces:

In [17]:
from search_methods import gradient_descent_random_sudoku

results = parallel_sudoku_search(gradient_descent_random_sudoku, fixed_squares, max_iterations=500)

  0%|          | 0/500 [00:00<?, ?it/s]

In [18]:
show_solution = True
for res in results:
    # Acá nos devuelve el booleano de si encontró la solución o no
    is_solution = res[0]
    # Este es el últimos estado encontrado en esta iteración
    last_state = res[1]
    # Este es el estado desde donde partio
    initial_state = res[2]
    # Este es el identificador de cual iteración se obtuvo la solución
    process_id = res[-1]

    if is_solution:
        if show_solution:
            print_state(last_state)
            show_solution = False
        print(f"En la iteración {process_id} se encontró la solución.")

*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*
En la iteración 62 se encontró la solución.
En la iteración 338 se encontró la solución.
En la iteración 475 se encontró la solución.


Vemos con este cambio, encontramos mas rápido soluciones y en mayor cantidad.

### Movernos en mesetas

La última variante que vamos a aplicar en este caso es permitir que, en el gradiente descendente estocástico, se pueda seleccionar vecinos en los que la diferencia entre el costo del vecino y el estado actual sea cero. Esto nos permitirá movernos en las mesetas para ver si podemos llegar a un precipicio y continuar reduciendo el costo.

Para esto, en la función `gradient_descent_random_sudoku()`, que está definida en `search_methods`, le agregamos un nuevo atributo llamado `move_in_zero`, que modifica una sola línea de la función:

```python
index_min_energy = [i for i, x in enumerate(neib_energy_list) if x <= 0]
```

La idea es quedarnos con todos los vecinos que mantengan o reduzcan el costo, y luego seleccionar uno de ellos.

In [19]:
from search_methods import gradient_descent_random_sudoku
from functools import partial

new_grad_desc = partial(gradient_descent_random_sudoku, move_in_zero=True)

results = parallel_sudoku_search(new_grad_desc, fixed_squares, max_iterations=500)

  0%|          | 0/500 [00:00<?, ?it/s]

In [20]:
show_solution = True
for res in results:
    # Acá nos devuelve el booleano de si encontró la solución o no
    is_solution = res[0]
    # Este es el últimos estado encontrado en esta iteración
    last_state = res[1]
    # Este es el estado desde donde partio
    initial_state = res[2]
    # Este es el identificador de cual iteración se obtuvo la solución
    process_id = res[-1]

    if is_solution:
        if show_solution:
            print_state(last_state)
            show_solution = False
        print(f"En la iteración {process_id} se encontró la solución.")

En esta corrida no se encontró solución. En este caso, moverse en la meseta resultó contraproducente.